# ModelCheckpoint与EarlyStopping深度解析

## 核心概念

### ModelCheckpoint
在训练过程中自动保存模型权重，支持多种保存策略：
- 保存所有epoch的模型
- 仅保存最佳性能的模型
- 按照指定频率保存
- 保存完整模型或仅保存权重

### EarlyStopping
当监控指标停止改善时自动终止训练，防止：
- 过拟合：验证集性能下降
- 资源浪费：不必要的训练迭代
- 时间浪费：性能已经收敛

## 应用场景

1. **长时间训练任务**: 防止训练中断导致进度丢失
2. **超参数搜索**: 快速淘汰表现不佳的配置
3. **模型选择**: 自动保存验证集上最佳的模型
4. **资源受限环境**: 在性能饱和时及时停止训练

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import json

## 1. 准备实验数据

使用CIFAR-10数据集演示，这是一个更复杂的图像分类任务。

In [ ]:
# 加载CIFAR-10数据集
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 数据预处理
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 转换标签为one-hot编码
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 为了快速演示，使用部分数据
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:2000]
y_test = y_test[:2000]

print(f'训练集形状: {x_train.shape}, 标签形状: {y_train.shape}')
print(f'测试集形状: {x_test.shape}, 标签形状: {y_test.shape}')

## 2. 构建卷积神经网络

构建一个适合CIFAR-10的CNN模型。

In [ ]:
def create_cnn_model():
    """
    创建卷积神经网络模型
    
    架构:
    - Conv2D (32) -> MaxPooling -> Conv2D (64) -> MaxPooling
    - Conv2D (128) -> GlobalAveragePooling
    - Dense (128) -> Dropout -> Dense (10)
    """
    model = keras.Sequential([
        # 第一个卷积块
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        
        # 第二个卷积块
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # 第三个卷积块
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # 全连接层
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model = create_cnn_model()
model.summary()

## 3. ModelCheckpoint详解

### 关键参数说明

- `filepath`: 保存路径，支持格式化字符串(如`model_{epoch:02d}_{val_loss:.2f}.h5`)
- `monitor`: 监控的指标名称
- `save_best_only`: 是否仅保存最佳模型
- `save_weights_only`: 是否仅保存权重(False则保存完整模型)
- `mode`: 'min'(指标越小越好)或'max'(指标越大越好)
- `save_freq`: 保存频率，'epoch'或整数(按batch数)
- `verbose`: 日志详细程度

In [ ]:
# 创建保存目录
checkpoint_dir = 'model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# 策略1: 保存最佳模型(基于验证准确率)
checkpoint_best_acc = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_accuracy.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# 策略2: 保存最佳模型(基于验证损失)
checkpoint_best_loss = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_loss.keras'),
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# 策略3: 保存每个epoch的模型
checkpoint_all = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_acc_{val_accuracy:.4f}.keras'),
    monitor='val_accuracy',
    save_best_only=False,
    save_freq='epoch',
    verbose=0
)

# 策略4: 仅保存权重
checkpoint_weights = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_weights.weights.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)

print('ModelCheckpoint回调函数配置完成')

## 4. EarlyStopping详解

### 关键参数说明

- `monitor`: 监控的指标名称
- `patience`: 容忍多少个epoch无改善
- `min_delta`: 最小改善幅度，小于此值视为无改善
- `mode`: 'min'或'max'
- `baseline`: 基准值，指标未达到基准值时不触发停止
- `restore_best_weights`: 是否恢复最佳权重
- `start_from_epoch`: 从第几个epoch开始监控

In [ ]:
# 策略1: 基于验证损失的早停
early_stop_loss = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,                    # 5个epoch无改善则停止
    min_delta=0.001,               # 改善小于0.001视为无改善
    mode='min',
    restore_best_weights=True,     # 恢复最佳权重
    verbose=1
)

# 策略2: 基于验证准确率的早停
early_stop_acc = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    min_delta=0.001,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

# 策略3: 设置基准值的早停
early_stop_baseline = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    baseline=0.60,                 # 准确率未达到60%不触发早停
    mode='max',
    restore_best_weights=True,
    verbose=1
)

print('EarlyStopping回调函数配置完成')

## 5. 训练模型 - 实验1: 基础配置

同时使用ModelCheckpoint和EarlyStopping。

In [ ]:
print('\n========== 实验1: 基础配置 ==========')
print('使用: 最佳准确率保存 + 验证损失早停\n')

model_exp1 = create_cnn_model()

# 组合回调函数
callbacks_exp1 = [
    checkpoint_best_acc,   # 保存最佳准确率模型
    early_stop_loss        # 基于损失的早停
]

history_exp1 = model_exp1.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,  # 测试用，实际训练可设为50
    validation_split=0.2,
    callbacks=callbacks_exp1,
    verbose=1
)

# 评估最终模型
test_loss, test_acc = model_exp1.evaluate(x_test, y_test, verbose=0)
print(f'\n最终模型 - 测试损失: {test_loss:.4f}, 测试准确率: {test_acc:.4f}')

## 6. 训练模型 - 实验2: 加载最佳模型

演示如何加载保存的最佳模型。

In [ ]:
print('\n========== 实验2: 加载最佳模型 ==========')

# 加载最佳准确率模型
best_acc_path = os.path.join(checkpoint_dir, 'best_accuracy.keras')
if os.path.exists(best_acc_path):
    best_model = keras.models.load_model(best_acc_path)
    print(f'成功加载模型: {best_acc_path}')
    
    # 在测试集上评估
    test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=0)
    print(f'\n最佳模型 - 测试损失: {test_loss:.4f}, 测试准确率: {test_acc:.4f}')
    
    # 与训练历史对比
    best_val_acc = max(history_exp1.history['val_accuracy'])
    print(f'训练时最佳验证准确率: {best_val_acc:.4f}')
else:
    print(f'未找到模型文件: {best_acc_path}')

## 7. 训练模型 - 实验3: 仅保存权重

演示权重保存和加载的方式。

In [ ]:
print('\n========== 实验3: 权重保存与加载 ==========')

model_exp3 = create_cnn_model()

callbacks_exp3 = [
    checkpoint_weights,    # 仅保存权重
    early_stop_acc         # 基于准确率的早停
]

history_exp3 = model_exp3.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_exp3,
    verbose=1
)

# 创建新模型并加载权重
weights_path = os.path.join(checkpoint_dir, 'best_weights.weights.h5')
if os.path.exists(weights_path):
    model_from_weights = create_cnn_model()
    model_from_weights.load_weights(weights_path)
    print(f'\n成功加载权重: {weights_path}')
    
    # 评估加载权重后的模型
    test_loss, test_acc = model_from_weights.evaluate(x_test, y_test, verbose=0)
    print(f'加载权重的模型 - 测试损失: {test_loss:.4f}, 测试准确率: {test_acc:.4f}')
else:
    print(f'未找到权重文件: {weights_path}')

## 8. 自定义保存逻辑

实现自定义的模型保存策略。

In [ ]:
class CustomModelCheckpoint(keras.callbacks.Callback):
    """
    自定义模型保存策略
    
    功能:
    1. 保存top-k个最佳模型
    2. 记录每个保存模型的详细信息
    3. 自动清理旧的模型文件
    """
    
    def __init__(self, save_dir, monitor='val_accuracy', mode='max', top_k=3):
        super().__init__()
        self.save_dir = save_dir
        self.monitor = monitor
        self.mode = mode
        self.top_k = top_k
        self.best_models = []  # 存储(score, epoch, filepath)元组
        self.metadata_file = os.path.join(save_dir, 'checkpoint_metadata.json')
        
        os.makedirs(save_dir, exist_ok=True)
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_score = logs.get(self.monitor)
        
        if current_score is None:
            return
        
        # 生成模型文件名
        filename = f'model_epoch_{epoch:02d}_{self.monitor}_{current_score:.4f}.keras'
        filepath = os.path.join(self.save_dir, filename)
        
        # 判断是否应该保存
        should_save = False
        if len(self.best_models) < self.top_k:
            should_save = True
        else:
            worst_score = min(self.best_models, key=lambda x: x[0])[0]
            if self.mode == 'max' and current_score > worst_score:
                should_save = True
            elif self.mode == 'min' and current_score < worst_score:
                should_save = True
        
        if should_save:
            # 保存模型
            self.model.save(filepath)
            self.best_models.append((current_score, epoch, filepath))
            
            # 按分数排序
            self.best_models.sort(key=lambda x: x[0], reverse=(self.mode == 'max'))
            
            # 如果超过top_k，删除最差的模型
            if len(self.best_models) > self.top_k:
                _, _, old_filepath = self.best_models.pop()
                if os.path.exists(old_filepath):
                    os.remove(old_filepath)
                    print(f'\n删除旧模型: {old_filepath}')
            
            print(f'\n保存模型: {filename} ({self.monitor}={current_score:.4f})')
            
            # 保存元数据
            self._save_metadata()
    
    def _save_metadata(self):
        """保存模型元数据到JSON文件"""
        metadata = {
            'monitor': self.monitor,
            'mode': self.mode,
            'top_k': self.top_k,
            'best_models': [
                {
                    'epoch': epoch,
                    'score': float(score),
                    'filepath': os.path.basename(filepath)
                }
                for score, epoch, filepath in self.best_models
            ]
        }
        
        with open(self.metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)

# 测试自定义回调
print('\n========== 实验4: 自定义模型保存 ==========')
print('保存top-3最佳模型\n')

custom_checkpoint_dir = os.path.join(checkpoint_dir, 'custom')
model_exp4 = create_cnn_model()

callbacks_exp4 = [
    CustomModelCheckpoint(
        save_dir=custom_checkpoint_dir,
        monitor='val_accuracy',
        mode='max',
        top_k=3
    )
]

history_exp4 = model_exp4.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_exp4,
    verbose=1
)

# 读取并显示元数据
metadata_path = os.path.join(custom_checkpoint_dir, 'checkpoint_metadata.json')
if os.path.exists(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    print('\n保存的模型元数据:')
    print(json.dumps(metadata, indent=2))

## 9. 分析训练历史

分析不同配置下的训练表现。

In [ ]:
def analyze_training_history(history, title):
    """
    分析训练历史
    """
    print(f'\n========== {title} ==========')
    
    # 提取指标
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    # 统计信息
    total_epochs = len(train_loss)
    best_epoch = np.argmax(val_acc) + 1
    best_val_acc = max(val_acc)
    final_val_acc = val_acc[-1]
    
    print(f'总训练轮数: {total_epochs}')
    print(f'最佳epoch: {best_epoch} (验证准确率: {best_val_acc:.4f})')
    print(f'最终验证准确率: {final_val_acc:.4f}')
    
    # 检查过拟合
    train_val_gap = train_acc[-1] - val_acc[-1]
    if train_val_gap > 0.1:
        print(f'\n警告: 存在过拟合迹象 (训练-验证准确率差距: {train_val_gap:.4f})')
    
    # 检查欠拟合
    if val_acc[-1] < 0.5:
        print('\n警告: 可能存在欠拟合，模型性能较差')
    
    return {
        'total_epochs': total_epochs,
        'best_epoch': best_epoch,
        'best_val_acc': best_val_acc,
        'final_val_acc': final_val_acc
    }

# 分析各个实验的训练历史
results = {}
results['exp1'] = analyze_training_history(history_exp1, '实验1分析')
results['exp3'] = analyze_training_history(history_exp3, '实验3分析')
results['exp4'] = analyze_training_history(history_exp4, '实验4分析')

# 对比结果
print('\n========== 实验对比 ==========')
for exp_name, result in results.items():
    print(f'{exp_name}: 最佳验证准确率={result["best_val_acc"]:.4f} @ epoch {result["best_epoch"]}')

## 10. 清理临时文件

In [ ]:
import shutil

# 清理测试产生的文件和目录
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    print(f'已删除目录: {checkpoint_dir}')

print('清理完成')

## 总结

### ModelCheckpoint最佳实践

1. **监控指标选择**:
   - 使用验证集指标而非训练集指标
   - 对于分类任务，优先监控准确率
   - 对于回归任务，监控损失或自定义指标

2. **保存策略**:
   - 长时间训练: 同时保存最佳模型和定期检查点
   - 快速实验: 仅保存最佳模型节省空间
   - 生产环境: 使用Keras格式(.keras)而非HDF5格式(.h5)

3. **文件命名**:
   - 包含关键信息: epoch、指标值、时间戳
   - 便于识别和排序

### EarlyStopping最佳实践

1. **patience设置**:
   - 数据量大、模型复杂: patience=5-10
   - 数据量小、模型简单: patience=3-5
   - 避免过小导致过早停止

2. **min_delta设置**:
   - 根据指标尺度调整
   - 准确率: 0.001-0.01
   - 损失: 0.0001-0.001

3. **restore_best_weights**:
   - 几乎总是设为True
   - 确保返回最佳性能的模型

### 组合使用建议

```python
callbacks = [
    # 保存最佳模型
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True
    ),
    # 早停机制
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
]
```

### 常见问题

1. **模型未保存**: 检查`save_best_only`和`monitor`配置
2. **过早停止**: 增大patience值
3. **磁盘空间不足**: 使用`save_best_only=True`或定期清理
4. **权重加载失败**: 确保模型架构完全一致